<a href="https://colab.research.google.com/github/masterplans-385/github-slideshow/blob/master/%E7%AB%B6%E9%A6%AC%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy import stats

# データ読み込み＆前処理

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/競馬/sample_race_data_2024-2022.txt", encoding='cp932')
df_header = pd.read_csv("/content/drive/MyDrive/競馬/成績データヘッダリスト2.txt", encoding='UTF-8')

columns = df_header.column_name.values
df.columns = columns

#タイムが「0」となっているデータを除外
df = df[df["タイム"] != 0]

#障害レースを除外
remove_grade_code = ["F","G","H"]
df = df[~df["グレードコード"].isin(remove_grade_code)]

#脚質「中団」「後方」を変更
df["脚質"] = df["脚質"].replace("中団", "差し")
df["脚質"] = df["脚質"].replace("後方", "追込")

In [ ]:
df.dtypes

In [ ]:
df.describe()

,年,月,日,回次,クラスコード,トラックコード,コーナー回数,距離,基準タイム,年齢,斤量,頭数,人気,タイム,通過順1角,通過順2角,通過順3角,通過順4角,上がり3Fタイム,上り3F順位,-3F差,Ave-3F,PCI,PCI3,馬体重,増減,賞金,血統登録番号,騎手コード,調教師コード,レースID（新)
count,129177.000000,129177.000000,129177.000000,129177.000000,129177.000000,129177.000000,125769.000000,129177.00000,0.0,129177.000000,129177.000000,129177.000000,129177.000000,129177.000000,129177.000000,129177.000000,129177.000000,129177.000000,129177.000000,125769.000000,125769.000000,125769.000000,125766.000000,125769.000000,129177.000000,116622.000000,129177.000000,1.291770e+05,129177.000000,129177.000000,1.291770e+05
mean,22.911439,5.989046,15.835915,2.458882,32.700550,1.384434,2.933211,1651.12760,NaN,3.636189,55.293477,14.309475,7.624864,102.217834,3.095249,3.527834,7.108781,7.152767,36.516824,7.404074,0.818038,36.574067,48.754957,50.301780,470.947615,0.288093,159.675755,2.019379e+09,1435.810020,1063.050419,2.022973e+17
std,0.791870,3.326092,8.452483,1.336898,40.122335,2.303881,1.081807,407.02266,NaN,1.311758,1.905949,2.581086,4.367680,28.101363,4.439916,4.570896,4.338732,4.308927,4.405350,4.397209,0.681074,1.386844,5.683291,4.857537,30.897021,6.632436,605.124737,1.506992e+06,1163.178472,247.018996,7.866110e+13
min,22.000000,1.000000,1.000000,1.000000,7.000000,0.000000,0.000000,1000.00000,NaN,2.000000,48.000000,5.000000,1.000000,53.900000,0.000000,0.000000,0.000000,1.000000,12.800000,1.000000,0.000000,31.800000,1.900000,33.930000,336.000000,-46.000000,0.000000,2.011101e+09,422.000000,359.000000,2.022011e+17
25%,22.000000,3.000000,9.000000,1.000000,7.000000,0.000000,2.000000,1400.00000,NaN,3.000000,54.000000,13.000000,4.000000,80.900000,0.000000,0.000000,3.000000,3.000000,35.300000,4.000000,0.300000,35.580000,45.000000,46.900000,450.000000,-4.000000,0.000000,2.018109e+09,1092.000000,1055.000000,2.022090e+17
50%,23.000000,6.000000,16.000000,2.000000,23.000000,1.000000,2.000000,1600.00000,NaN,3.000000,55.000000,15.000000,7.000000,101.800000,0.000000,1.000000,7.000000,7.000000,36.900000,7.000000,0.700000,36.600000,48.600000,49.930000,472.000000,0.000000,0.000000,2.019107e+09,1154.000000,1102.000000,2.023051e+17
75%,24.000000,9.000000,23.000000,3.000000,43.000000,1.000000,4.000000,1800.00000,NaN,4.000000,57.000000,16.000000,11.000000,117.100000,6.000000,7.000000,10.000000,10.000000,38.500000,11.000000,1.200000,37.590000,52.600000,53.430000,492.000000,4.000000,140.000000,2.020107e+09,1185.000000,1148.000000,2.024012e+17
max,24.000000,12.000000,31.000000,6.000000,195.000000,8.000000,6.000000,3600.00000,NaN,12.000000,65.000000,18.000000,18.000000,252.600000,18.000000,18.000000,18.000000,18.000000,98.400000,18.000000,21.300000,48.900000,80.400000,70.470000,630.000000,48.000000,50000.000000,2.022110e+09,5664.000000,5782.000000,2.024093e+17


In [ ]:
df.isna().sum()

,0
年,0
月,0
日,0
回次,0
場所,0
日次,0
レース名,0
クラス名,0
クラスコード,0
グレードコード,95222


In [ ]:
zero_time_df = df[df["タイム"] == 0]
zero_time_df

,年,月,日,回次,場所,日次,レース名,クラス名,クラスコード,グレードコード,芝・ダ,トラックコード,コーナー回数,距離,コース区分,馬場状態,基準タイム,馬名,性別,年齢,脚質,父馬名,母馬名,母の父馬名,父タイプ,母父タイプ,騎手,斤量,頭数,枠番,馬番,確定着順,着差タイム,人気,タイム,タイムS,通過順1角,通過順2角,通過順3角,通過順4角,上がり3Fタイム,上り3F順位,-3F差,Ave-3F,PCI,PCI3,馬体重,増減,調教師,所属,賞金,血統登録番号,騎手コード,調教師コード,レースID（新),生産者名


## データフレーム作成用関数

In [ ]:
#データフレーム作成
def create_dataframe(df, surface_type, distance=None, arrival=None, place=None, class_code=None, track_condition=None):
  #フィルタ条件を定義
  conditions = {
      "芝・ダ": surface_type,
      "距離": distance,
      "確定着順": arrival,
      "場所": place,
      "クラスコード": class_code,
      "馬場状態": track_condition
  }

  #初期フィルタリング
  df_filtered = df

  #Noneでない条件を適用
  for col, val in conditions.items():
    if val is not None:
      df_filtered = df_filtered[df_filtered[col] == val]

  df_time = df_filtered[["場所", "レース名", "クラス名", "クラスコード", "グレードコード", "芝・ダ", "距離", "馬場状態", "馬名", "性別", "確定着順", 'タイム', "上がり3Fタイム"]]

  return df_time

#タイム分析

## 関数

### タイムの単位変換関数

In [ ]:
def seconds_to_min_sec(seconds):
  minutes = int(seconds // 60)
  remaining_seconds = int(seconds % 60)
  return f"{minutes}分{remaining_seconds:02d}秒"

### クラスコード変換関数

In [ ]:
#クラスコード------------
# 7 未勝利　# 11 未出走　# 15 新馬　# 19 400万下　# 23 500万下　# 39 900万下
# 43 1000万下　# 63 1500万下　# 67 1600万下　# 131 重賞以外のオープン　# 147 グレード無し重賞
# 163 G3　# 179 G2　# 195 G1

In [ ]:
#クラスコード変換関数
def classname_to_code(class_name):
  #変換条件の定義
  class_cat = {
      "未勝利":7,
      "未出走":11,
      "新馬":15,
      "400":19,
      "1勝":23,
      "2勝":43,
      "900":39,
      "1500":63,
      "3勝":67,
      "オープン":131,
      "グレード無し重賞":147,
      "L":115,
      "G3":163,
      "G2":179,
      "G1":195,
  }

  return class_cat.get(class_name)

### 共通プロット作成・統計計算関数

In [ ]:
#タイム共通プロット作成・統計計算関数
def create_plots_and_stats(df, group_cols, x_col, violin_color_col, hover_data_cols, title, category_orders=None):
  #平均タイムの計算
  ave_time = df.groupby(group_cols)["タイム"].mean().round(2).reset_index().rename(columns={"タイム":"平均タイム"})
  ave_time["タイム(分)"] = ave_time["平均タイム"].apply(seconds_to_min_sec)
  #標準偏差の計算
  std_time = df.groupby(group_cols)["タイム"].std().round(2).reset_index().rename(columns={"タイム":"標準偏差"})
  #最小値の計算
  min_time = df.groupby(group_cols)["タイム"].min().round(2).reset_index().rename(columns={"タイム":"最小値"})

  #データフレームの結合
  df_stat = pd.merge(ave_time, std_time, on=group_cols)
  df_stat = pd.merge(df_stat, min_time, on=group_cols)

  #バイオリンプロットの作成
  fig_violin = px.violin(
      df,
      x=x_col,
      y="タイム",
      color=violin_color_col,
      hover_data=hover_data_cols,
      box=True,
      points="all",
      title=title,
      category_orders=category_orders
  )

  #余白調整
  fig_violin.update_layout(
      margin = dict(l=50, r=50, t=60, b=20)
  )

  #バイオリンプロットを表示
  fig_violin.show()

  #平均タイム表
  table = go.Table(
    header=dict(values=list(df_stat.columns),fill_color="paleturquoise",align="left"),
    cells=dict(values=[df_stat[col] for col in df_stat.columns],fill_color="lavender", align="left")
  )

  fig_table = go.Figure(data=[table])

  #表の余白を調整
  fig_table.update_layout(
      margin = dict(l=50, r=50, t=60, b=20)
  )

  fig_table.show()

In [ ]:
#上がり3Fタイム共通プロット作成・統計計算関数
def create_3Ftime_plots_and_stats(df, group_cols, x_col, violin_color_col, hover_data_cols, title, category_orders=None):
  #平均タイムの計算
  ave_time = df.groupby(group_cols)["上がり3Fタイム"].mean().round(2).reset_index().rename(columns={"タイム":"上がり3Fタイム"})
  #標準偏差の計算
  std_time = df.groupby(group_cols)["上がり3Fタイム"].std().round(2).reset_index().rename(columns={"上がり3Fタイム":"標準偏差"})
  #最小値の計算
  min_time = df.groupby(group_cols)["上がり3Fタイム"].min().round(2).reset_index().rename(columns={"上がり3Fタイム":"最小値"})

  #データフレームの結合
  df_stat = pd.merge(ave_time, std_time, on=group_cols)
  df_stat = pd.merge(df_stat, min_time, on=group_cols)

  #バイオリンプロットの作成
  fig_violin = px.violin(
      df,
      x=x_col,
      y="上がり3Fタイム",
      color=violin_color_col,
      hover_data=hover_data_cols,
      box=True,
      points="all",
      title=title,
      category_orders=category_orders
  )

  #余白調整
  fig_violin.update_layout(
      margin = dict(l=50, r=50, t=60, b=20)
  )

  #バイオリンプロットを表示
  fig_violin.show()

  #平均タイム表
  table = go.Table(
    header=dict(values=list(df_stat.columns),fill_color="paleturquoise",align="left"),
    cells=dict(values=[df_stat[col] for col in df_stat.columns],fill_color="lavender", align="left")
  )

  fig_table = go.Figure(data=[table])

    #表の余白を調整
  fig_table.update_layout(
      margin = dict(l=50, r=50, t=60, b=20)
  )
  fig_table.show()

### 共通フィルタリング関数

In [ ]:
#共通フィルタリング関数
def apply_conditions(df, surface_type, distance=None, arrival=None, place=None, class_name=None, track_condition=None):
  #フィルタ条件を定義
  class_code = None
  if class_name is not None:
    class_code = classname_to_code(class_name)
  conditions = {
      "芝・ダ": surface_type,
      "距離": distance,
      "確定着順": arrival,
      "クラスコード": class_code,
      "場所": place,
      "馬場状態": track_condition
  }

  #Noneでない条件を適用
  df_filtered = df
  for col, val in conditions.items():
    if val is not None:
      df_filtered = df_filtered[df_filtered[col] == val]

  return df_filtered

### タイムバイオリンプロット・統計表描画用関数

#### 距離別タイム

In [ ]:
#距離別コースタイムバイオリンプロット・統計表描画用関数(mode タイム→コースタイム、3F→上がり3Fタイム)
def distance_time_plots(df, surface_type, mode, arrival=None, class_name=None, place=None, track_condition=None):
  #フィルタリング
  df_filtered = apply_conditions(df, surface_type, arrival=arrival, class_name=class_name, place=place, track_condition=track_condition)

  if mode == "タイム":
    #コースタイム
    #タイトル設定
    violin_title = f"競馬場：{place}({surface_type}) クラス：{class_name} 馬場状態：{track_condition} タイム"

    #プロット・統計作成
    create_plots_and_stats(
        df_filtered,
        group_cols=["距離", "芝・ダ"],
        x_col="距離",
        violin_color_col="距離",
        hover_data_cols=["馬名", "レース名"],
        title=violin_title
    )
  elif mode == "3F":
    #上がり3Fタイム
    #タイトル設定
    violin_title = f"競馬場：{place}({surface_type}) クラス：{class_name} 馬場状態：{track_condition} 上がり3Fタイム"

    #プロット・統計作成
    create_3Ftime_plots_and_stats(
        df_filtered,
        group_cols=["距離", "芝・ダ"],
        x_col="距離",
        violin_color_col="距離",
        hover_data_cols=["馬名", "レース名"],
        title=violin_title
  )

#### 競馬場別タイム

In [ ]:
#競馬場別コースタイムバイオリンプロット・統計表描画用関数(mode タイム→コースタイム、3F→上がり3Fタイム)
def place_time_plots(df, distance, surface_type, mode, arrival=None, class_name=None, place=None, track_condition=None):
  #フィルタリング
  df_filtered = apply_conditions(df, surface_type, distance=distance, arrival=arrival, class_name=class_name, track_condition=track_condition)

  if mode == "タイム":
    #コースタイム
    #タイトル設定
    violin_title = f"{distance}M({surface_type}) クラス:{class_name} タイム"

    #プロット・統計作成
    create_plots_and_stats(
        df_filtered,
        group_cols=["場所", "芝・ダ"],
        x_col="場所",
        violin_color_col="場所",
        hover_data_cols=["馬名", "レース名"],
        title=violin_title
    )

  if mode == "3F":
    #上がり3Fタイム
    #タイトル設定
    violin_title = f"{distance}M({surface_type}) クラス:{class_name} 上がり3Fタイム"

    #プロット・統計作成
    create_3Ftime_plots_and_stats(
        df_filtered,
        group_cols=["場所", "芝・ダ"],
        x_col="場所",
        violin_color_col="場所",
        hover_data_cols=["馬名", "レース名"],
        title=violin_title
    )

#### クラス別タイム

In [ ]:
#クラス別コースタイムバイオリンプロット・統計表描画用関数(mode タイム→コースタイム、3F→上がり3Fタイム)
def class_time_plots(df, distance, surface_type, mode, arrival=None, place=None, track_condition=None):
  #フィルタリング
  df_filtered = apply_conditions(df, surface_type, distance=distance, arrival=arrival, place=place, track_condition=track_condition)

  if mode == "タイム":
    #コースタイム
    #タイトル設定
    violin_title = f"競馬場:{place} {distance}M ({surface_type})"

    #プロット・統計作成(クラス順序を指定)
    class_order = ["新馬", "未勝利", "1勝", "2勝", "3勝", "ｵｰﾌﾟﾝ", "OP(L)", "Ｇ３", "Ｇ２", "Ｇ１"]
    create_plots_and_stats(
        df_filtered,
        group_cols=["クラス名", "芝・ダ"],
        x_col="クラス名",
        violin_color_col="クラス名",
        hover_data_cols=["馬名", "レース名"],
        title=violin_title,
        category_orders={"クラス名": class_order}
    )

  elif mode == "3F":
    #上がり3Fタイム
    #タイトル設定
    violin_title = f"競馬場:{place} {distance}M ({surface_type}) 上がり3Fタイム"

    #プロット・統計作成(クラス順序を指定)
    class_order = ["新馬", "未勝利", "1勝", "2勝", "3勝", "ｵｰﾌﾟﾝ", "OP(L)", "Ｇ３", "Ｇ２", "Ｇ１"]
    create_3Ftime_plots_and_stats(
        df_filtered,
        group_cols=["クラス名", "芝・ダ"],
        x_col="クラス名",
        violin_color_col="クラス名",
        hover_data_cols=["馬名", "レース名"],
        title=violin_title,
        category_orders={"クラス名": class_order}
    )

#### 馬場状態別タイム

In [ ]:
#馬場別コースタイムバイオリンプロット・統計表描画用関数(mode タイム→コースタイム、3F→上がり3Fタイム)
def condition_time_plots(df, distance, surface_type, mode, arrival=None, place=None, class_name=None):
  #フィルタリング
  df_filtered = apply_conditions(df, surface_type, distance=distance, arrival=arrival, place=place, class_name=class_name)

  if mode == "タイム":
    #コースタイム
    #タイトル設定
    violin_title = f"競馬場{place} {distance}M ({surface_type}){class_name}クラス"

    #プロット・統計作成(馬場状態順序を指定)
    class_order = ["良", "稍", "重", "不"]
    create_plots_and_stats(
        df_filtered,
        group_cols=["馬場状態", "芝・ダ"],
        x_col="馬場状態",
        violin_color_col="馬場状態",
        hover_data_cols=["馬名", "レース名"],
        title=violin_title,
        category_orders={"馬場状態": class_order}
    )

  if mode == "3F":
    #上がり3Fタイム
    #タイトル設定
    violin_title = f"競馬場{place} {distance}M({surface_type}){class_name}クラス 上がり3Fタイム"

    #プロット・統計作成(馬場状態順序を指定)
    class_order = ["良", "稍", "重", "不"]
    create_3Ftime_plots_and_stats(
        df_filtered,
        group_cols=["馬場状態", "芝・ダ"],
        x_col="馬場状態",
        violin_color_col="馬場状態",
        hover_data_cols=["馬名", "レース名"],
        title=violin_title,
        category_orders={"馬場状態": class_order}
    )

## タイムグラフ描画

### 距離別タイムグラフ描画

In [ ]:
distance_time_plots(df, mode="タイム", surface_type="芝", arrival=1,track_condition="良", place="東京", class_name="G2")

### 競馬場別タイムグラフ描画

In [ ]:
place_time_plots(df, mode="タイム", distance=1200, surface_type="芝", arrival=1,class_name="G1")

### クラス別タイムグラフ描画

In [ ]:
class_time_plots(df, mode="タイム", distance=2000, surface_type="芝", arrival=1, track_condition="良")

### 馬場状態別タイムグラフ描画

In [ ]:
condition_time_plots(df, mode="タイム", distance=2000, surface_type="ダ", arrival=1)

### タイム分散分析

In [ ]:
df_1 = create_dataframe(df, surface_type="芝", distance=1200, arrival=1, track_condition="良")
df_2 = create_dataframe(df, surface_type="芝", distance=1200, arrival=1, track_condition="稍")

data_1 = df_1["タイム"]
data_2 = df_2["タイム"]

f_statistic, p_value = stats.f_oneway(data_1, data_2)
print(f"F値: {f_statistic:.3f}, p値:{p_value:.3f}")

F値: 88.526, p値:0.000


# クロス分析

## 関数

### 枠番別クロス集計関数

In [ ]:
#枠番別クロス集計表描画関数
def create_crosstable_wakuban(df,surface_type,mode,distance=None, place=None, class_name=None, track_condition=None):
  #フィルタリング
  df_filtered = apply_conditions(df, surface_type=surface_type, distance=distance, place=place, class_name=class_name, track_condition=track_condition)

  #クロス集計表の作成
  pivot_table = pd.pivot_table(
      df_filtered,
      index =  "確定着順",
      columns = "枠番",
      aggfunc = "size",
      fill_value = 0,
  )

  if mode == "count":
    #勝利数モード

    #行の合計を追加
    pivot_table["合計"] = pivot_table.sum(axis=1)

    #列の合計を追加
    column_totals = pivot_table.sum(axis=0)
    column_totals.name = "合計"

    #合計を追加したデータフレーム
    pivot_table_with_totals = pd.concat([pivot_table, pd.DataFrame(column_totals).T])

    #Tableを作成
    fig = go.Figure(data=[go.Table(
        header = dict(
            values = ["確定着順"] + list(pivot_table_with_totals.columns),
            fill_color = "paleturquoise",
            align = "center"
        ),
        cells = dict(
            values = [pivot_table_with_totals.index] + [pivot_table_with_totals[col] for col in pivot_table_with_totals.columns],
            fill_color = "lavender",
            align = "center"
        )
    )])

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 枠番別勝利数クロス集計",
        title_x = 0.5
    )

  elif mode == "row_ratio":
    #勝率モード(行パーセント)
    # 各行の合計を求めて割合を計算
    row_sums = pivot_table.sum(axis=1)
    pivot_table_percentage = pivot_table.div(row_sums, axis=0).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    #8枠以降の番号を削除
    pivot_table_percentage = pivot_table_percentage.loc[:, pivot_table_percentage.columns <= 8]

    #Tableを作成
    fig = go.Figure(data=[go.Table(
        header = dict(
            values = ["確定着順"] + list(pivot_table_percentage.columns),
            fill_color = "paleturquoise",
            align = "center"
        ),
        cells = dict(
            values = [pivot_table_percentage.index] + [pivot_table_percentage[col] for col in pivot_table_percentage.columns],
            fill_color = "lavender",
            align = "center"
        )
    )])

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 枠番別勝率クロス集計(行パーセント)",
        title_x = 0.5
    )

  elif mode == "col_ratio":
    #勝率モード(列パーセント)
    # 各行の合計を求めて割合を計算
    col_sums = pivot_table.sum(axis=0)
    pivot_table_percentage = pivot_table.div(col_sums, axis=1).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    #8枠以降の番号を削除
    pivot_table_percentage = pivot_table_percentage.loc[:, pivot_table_percentage.columns <= 8]

    #Tableを作成
    fig = go.Figure(data=[go.Table(
        header = dict(
            values = ["確定着順"] + list(pivot_table_percentage.columns),
            fill_color = "paleturquoise",
            align = "center"
        ),
        cells = dict(
            values = [pivot_table_percentage.index] + [pivot_table_percentage[col] for col in pivot_table_percentage.columns],
            fill_color = "lavender",
            align = "center"
        )
    )])

    #タイトル追加
    fig.update_layout(
        title = f"{place}競馬場({surface_type}){distance}M{class_name}クラス{track_condition} 枠番別勝率クロス集計(列パーセント)",
        title_x = 0.5
    )

  else:
    raise ValueError("Invalid mode. Use 'count' for 勝利数 or 'ratio' for 勝率")

  fig.show()

### 枠番別勝利数ヒートマップ描画関数

In [ ]:
#枠番ヒートマップ
def create_heatmap_wakuban(df,surface_type, mode, distance=None, place=None, class_name=None, track_condition=None):
  #フィルタリング
  df_filtered = apply_conditions(df, surface_type=surface_type, distance=distance, place=place, class_name=class_name, track_condition=track_condition)

  #クロス集計表の作成
  pivot_table = pd.pivot_table(
      df_filtered,
      index =  "確定着順",
      columns = "枠番",
      aggfunc = "size",
      fill_value = 0,
  )

  if mode == "count":
    #勝利数モード
    z_values = pivot_table.values #セルの値
    x_values = pivot_table.columns #枠番
    y_values = pivot_table.index #確定着順

    #ヒートマップを作成
    fig = go.Figure(data=go.Heatmap(
        z = z_values,
        x = x_values,
        y = y_values,
        colorscale = "Blues",
        text = z_values, #セル内の値
        texttemplate = "%{text}", #セル内に値を表示
        hoverinfo = "text"
    ))

    #y軸を逆順にする
    fig.update_yaxes(autorange="reversed")

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 枠番別勝利数クロス集計",
        xaxis = dict(
            title = "枠番",
            side = "top"
        ),
        yaxis_title = "確定着順",
    )

  elif mode == "row_ratio":
    #勝率モード(行パーセント)

    # 各行の合計を求めて割合を計算
    row_sums = pivot_table.sum(axis=1)
    pivot_table_percentage = pivot_table.div(row_sums, axis=0).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    #8枠以降の番号を削除
    pivot_table_percentage = pivot_table_percentage.loc[:, pivot_table_percentage.columns <= 8]

    z_values = pivot_table_percentage.values #セルの値
    x_values = pivot_table_percentage.columns #枠番
    y_values = pivot_table_percentage.index #確定着順

    #ヒートマップを作成
    fig = go.Figure(data=go.Heatmap(
        z = z_values,
        x = x_values,
        y = y_values,
        colorscale = "Blues",
        text = z_values, #セル内の値
        texttemplate = "%{text}", #セル内に値を表示
        hoverinfo = "text"
    ))

    #y軸を逆順にする
    fig.update_yaxes(autorange="reversed")

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 枠番別勝率クロス集計(行パーセント)",
        xaxis = dict(
            title = "枠番",
            side = "top"
        ),
        yaxis_title = "確定着順",
    )

  elif mode == "col_ratio":
    #勝率モード(列パーセント)

    # 各行の合計を求めて割合を計算
    col_sums = pivot_table.sum(axis=0)
    pivot_table_percentage = pivot_table.div(col_sums, axis=1).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    #8枠以降の番号を削除
    pivot_table_percentage = pivot_table_percentage.loc[:, pivot_table_percentage.columns <= 8]

    z_values = pivot_table_percentage.values #セルの値
    x_values = pivot_table_percentage.columns #枠番
    y_values = pivot_table_percentage.index #確定着順

    #ヒートマップを作成
    fig = go.Figure(data=go.Heatmap(
        z = z_values,
        x = x_values,
        y = y_values,
        colorscale = "Blues",
        text = z_values, #セル内の値
        texttemplate = "%{text}", #セル内に値を表示
        hoverinfo = "text"
    ))

    #y軸を逆順にする
    fig.update_yaxes(autorange="reversed")

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 枠番別勝率クロス集計(列パーセント)",
        xaxis = dict(
            title = "枠番",
            side = "top"
        ),
        yaxis_title = "確定着順",
    )


  else:
    raise ValueError("Invalid mode. Use 'count' for 勝利数 or 'ratio' for 勝率")

  fig.show()

### 脚質別勝利数クロス集計表描画関数

In [ ]:
#脚質別クロス集計表描画関数
def create_crosstable_kyakushitsu(df,surface_type, mode, distance=None, place=None, class_name=None, track_condition=None):
  #フィルタリング
  df_filtered = apply_conditions(df, surface_type=surface_type, distance=distance, place=place, class_name=class_name, track_condition=track_condition)

  #クロス集計表の作成
  pivot_table = pd.pivot_table(
      df_filtered,
      index =  "確定着順",
      columns = "脚質",
      aggfunc = "size",
      fill_value = 0,
  )

  #列の順序
  column_order = ["逃げ", "先行", "差し", "追込", "ﾏｸﾘ"]

  #列の並び替え
  pivot_table = pivot_table.reindex(columns = column_order, fill_value = 0 )

  if mode == "count":
    #勝利数モード

    #行の合計を追加
    pivot_table["合計"] = pivot_table.sum(axis=1)

    #列の合計を追加
    column_totals = pivot_table.sum(axis=0)
    column_totals.name = "合計"

    #合計を追加したデータフレーム
    pivot_table_with_totals = pd.concat([pivot_table, pd.DataFrame(column_totals).T])

    #Tableを作成
    fig = go.Figure(data=[go.Table(
        header = dict(
            values = ["確定着順"] + list(pivot_table_with_totals.columns),
            fill_color = "paleturquoise",
            align = "center"
        ),
        cells = dict(
            values = [pivot_table_with_totals.index] + [pivot_table_with_totals[col] for col in pivot_table_with_totals.columns],
            fill_color = "lavender",
            align = "center"
        )
    )])

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition}脚質別勝利数クロス集計",
        title_x = 0.5
    )

  elif mode == "row_ratio":
    #勝率モード(行パーセント)
    # 各行の合計を求めて割合を計算
    row_sums = pivot_table.sum(axis=1)
    pivot_table_percentage = pivot_table.div(row_sums, axis=0).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    #Tableを作成
    fig = go.Figure(data=[go.Table(
        header = dict(
            values = ["確定着順"] + list(pivot_table_percentage.columns),
            fill_color = "paleturquoise",
            align = "center"
        ),
        cells = dict(
            values = [pivot_table_percentage.index] + [pivot_table_percentage[col] for col in pivot_table_percentage.columns],
            fill_color = "lavender",
            align = "center"
        )
    )])

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 脚質別勝率クロス集計(行パーセント)",
        title_x = 0.5
    )

  elif mode == "col_ratio":
    #勝率モード(列パーセント)
    # 各行の合計を求めて割合を計算
    col_sums = pivot_table.sum(axis=0)
    pivot_table_percentage = pivot_table.div(col_sums, axis=1).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    #Tableを作成
    fig = go.Figure(data=[go.Table(
        header = dict(
            values = ["確定着順"] + list(pivot_table_percentage.columns),
            fill_color = "paleturquoise",
            align = "center"
        ),
        cells = dict(
            values = [pivot_table_percentage.index] + [pivot_table_percentage[col] for col in pivot_table_percentage.columns],
            fill_color = "lavender",
            align = "center"
        )
    )])

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 脚質別勝率クロス集計(列パーセント)",
        title_x = 0.5
    )

  else:
    raise ValueError("Invalid mode. Use 'count' for 勝利数 or 'ratio' for 勝率")

  fig.show()

### 脚質別ヒートマップ描画関数

In [ ]:
#脚質別ヒートマップ描画関数
def create_heatmap_kyakushitsu(df,surface_type, mode, distance=None, place=None, class_name=None, track_condition=None):
  #フィルタリング
  df_filtered = apply_conditions(df, surface_type=surface_type, distance=distance, place=place, class_name=class_name, track_condition=track_condition)

  #クロス集計表の作成
  pivot_table = pd.pivot_table(
      df_filtered,
      index =  "確定着順",
      columns = "脚質",
      aggfunc = "size",
      fill_value = 0,
  )

  #列の順序
  column_order = ["逃げ", "先行", "差し", "追込", "ﾏｸﾘ"]

  #列の並び替え
  pivot_table = pivot_table.reindex(columns = column_order, fill_value = 0 )

  if mode == "count":
    #勝利数モード
    z_values = pivot_table.values #セルの値
    x_values = pivot_table.columns #脚質
    y_values = pivot_table.index #確定着順

    #ヒートマップを作成
    fig = go.Figure(data=go.Heatmap(
        z = z_values,
        x = x_values,
        y = y_values,
        colorscale = "Blues",
        text = z_values, #セル内の値
        texttemplate = "%{text}", #セル内に値を表示
        hoverinfo = "text"
    ))

    #y軸を逆順にする
    fig.update_yaxes(autorange="reversed")

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 脚質別勝利数クロス集計",
        xaxis = dict(
            title = "脚質",
            side = "top"
        ),
        yaxis_title = "確定着順",
    )

  elif mode == "row_ratio":
    #勝率モード(行パーセント)

    # 各行の合計を求めて割合を計算
    row_sums = pivot_table.sum(axis=1)
    pivot_table_percentage = pivot_table.div(row_sums, axis=0).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    z_values = pivot_table_percentage.values #セルの値
    x_values = pivot_table_percentage.columns #脚質
    y_values = pivot_table_percentage.index #確定着順

    #ヒートマップを作成
    fig = go.Figure(data=go.Heatmap(
        z = z_values,
        x = x_values,
        y = y_values,
        colorscale = "Blues",
        text = z_values, #セル内の値
        texttemplate = "%{text}", #セル内に値を表示
        hoverinfo = "text"
    ))

    #y軸を逆順にする
    fig.update_yaxes(autorange="reversed")

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 脚質別勝率クロス集計(行パーセント)",
        xaxis = dict(
            title = "脚質",
            side = "top"
        ),
        yaxis_title = "確定着順",
    )

  elif mode == "col_ratio":
    #勝率モード(列パーセント)

    # 各行の合計を求めて割合を計算
    col_sums = pivot_table.sum(axis=0)
    pivot_table_percentage = pivot_table.div(col_sums, axis=1).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    z_values = pivot_table_percentage.values #セルの値
    x_values = pivot_table_percentage.columns #脚質
    y_values = pivot_table_percentage.index #確定着順

    #ヒートマップを作成
    fig = go.Figure(data=go.Heatmap(
        z = z_values,
        x = x_values,
        y = y_values,
        colorscale = "Blues",
        text = z_values, #セル内の値
        texttemplate = "%{text}", #セル内に値を表示
        hoverinfo = "text"
    ))

    #y軸を逆順にする
    fig.update_yaxes(autorange="reversed")

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 脚質別勝率クロス集計(列パーセント)",
        xaxis = dict(
            title = "脚質",
            side = "top"
        ),
        yaxis_title = "確定着順",
    )

  else:
    raise ValueError("Invalid mode. Use 'count' for 勝利数 or 'ratio' for 勝率")

  fig.show()

## 勝利数クロス集計表・ヒートマップ描画

### 枠番別クロス集計表

In [ ]:
#枠番別クロス集計 勝利数:mode=count, 勝率:mode=row_ratio(行パーセント),col_ratio(列パーセント)
create_crosstable_wakuban(df,surface_type="芝", mode="count", class_name="G1")
create_crosstable_wakuban(df,surface_type="芝", mode="col_ratio")

### 脚質別クロス集計表

In [ ]:
#脚質別クロス集計表 勝利数:mode=count, 勝率:mode=row_ratio(行パーセント),col_ratio(列パーセント)
# create_crosstable_kyakushitsu(df,surface_type="芝",place="東京", mode="count")
create_crosstable_kyakushitsu(df,surface_type="芝",mode="row_ratio")
create_crosstable_kyakushitsu(df,surface_type="芝",place="東京",mode="row_ratio")
create_crosstable_kyakushitsu(df,surface_type="芝",place="中山",mode="row_ratio")

### 枠番別ヒートマップ

In [ ]:
#枠番別ヒートマップ 勝利数:mode=count, 勝率:mode=row_ratio(行パーセント),col_ratio(列パーセント)
create_heatmap_wakuban(df,surface_type="芝", mode="count", class_name="G1")
create_heatmap_wakuban(df,surface_type="芝", mode="col_ratio", class_name="G1")

### 脚質別ヒートマップ

In [ ]:
#脚質別ヒートマップ 勝利数:mode=count, 勝率:mode=row_ratio(行パーセント),col_ratio(列パーセント)
create_heatmap_kyakushitsu(df,surface_type="芝", mode="count")
create_heatmap_kyakushitsu(df,surface_type="芝", mode="col_ratio")

# テスト用

In [ ]:
#枠番別クロス集計表描画関数
def create_crosstable_wakuban(df,surface_type,mode,distance=None, place=None, class_name=None, track_condition=None):
  #フィルタリング
  df_filtered = apply_conditions(df, surface_type=surface_type, distance=distance, place=place, class_name=class_name, track_condition=track_condition)

  #クロス集計表の作成
  pivot_table = pd.pivot_table(
      df_filtered,
      index =  "確定着順",
      columns = "枠番",
      aggfunc = "size",
      fill_value = 0,
  )

  #行の合計を追加
  pivot_table["合計"] = pivot_table.sum(axis=1)

  #列の合計を追加
  column_totals = pivot_table.sum(axis=0)
  column_totals.name = "合計"

  #合計を追加したデータフレーム
  pivot_table = pd.concat([pivot_table, pd.DataFrame(column_totals).T])

  if mode == "count":
    #勝利数モード

    #Tableを作成
    fig = go.Figure(data=[go.Table(
        header = dict(
            values = ["確定着順"] + list(pivot_table.columns),
            fill_color = "paleturquoise",
            align = "center"
        ),
        cells = dict(
            values = [pivot_table.index] + [pivot_table[col] for col in pivot_table.columns],
            fill_color = "lavender",
            align = "center"
        )
    )])

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 枠番別勝利数クロス集計",
        title_x = 0.5
    )

  elif mode == "row_ratio":
    #勝率モード(行パーセント)

    # 合計行と合計列を一時的に除外
    pivot_table = pivot_table.drop(index="合計", columns="合計")

    # 各行の合計を求めて割合を計算
    row_sums = pivot_table.sum(axis=1)
    pivot_table_percentage = pivot_table.div(row_sums, axis=0).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    #8枠以降の番号を削除
    # pivot_table_percentage = pivot_table_percentage.loc[:, pivot_table_percentage.columns <= 8]

    #Tableを作成
    fig = go.Figure(data=[go.Table(
        header = dict(
            values = ["確定着順"] + list(pivot_table_percentage.columns),
            fill_color = "paleturquoise",
            align = "center"
        ),
        cells = dict(
            values = [pivot_table_percentage.index] + [pivot_table_percentage[col] for col in pivot_table_percentage.columns],
            fill_color = "lavender",
            align = "center"
        )
    )])

    #タイトル追加
    fig.update_layout(
        title = f"競馬場:{place}({surface_type}) {distance}M クラス:{class_name} 馬場状態:{track_condition} 枠番別勝率クロス集計(行パーセント)",
        title_x = 0.5
    )

  elif mode == "col_ratio":
    #勝率モード(列パーセント)

    # 合計行と合計列を一時的に除外
    pivot_table = pivot_table.drop(index="合計", columns="合計")

    # 各行の合計を求めて割合を計算
    col_sums = pivot_table.sum(axis=0)
    pivot_table_percentage = pivot_table.div(col_sums, axis=1).round(4)  # ゼロを避ける
    pivot_table_percentage = (pivot_table_percentage * 100).round(2)

    #8枠以降の番号を削除
    # pivot_table_percentage = pivot_table_percentage.loc[:, pivot_table_percentage.columns <= 8]

    #Tableを作成
    fig = go.Figure(data=[go.Table(
        header = dict(
            values = ["確定着順"] + list(pivot_table_percentage.columns),
            fill_color = "paleturquoise",
            align = "center"
        ),
        cells = dict(
            values = [pivot_table_percentage.index] + [pivot_table_percentage[col] for col in pivot_table_percentage.columns],
            fill_color = "lavender",
            align = "center"
        )
    )])

    #タイトル追加
    fig.update_layout(
        title = f"{place}競馬場({surface_type}){distance}M{class_name}クラス{track_condition} 枠番別勝率クロス集計(列パーセント)",
        title_x = 0.5
    )

  else:
    raise ValueError("Invalid mode. Use 'count' for 勝利数 or 'ratio' for 勝率")

  fig.show()

In [ ]:
create_crosstable_wakuban(df,surface_type="芝",mode="col_ratio")